In [212]:
import math
import random

In [ ]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("zhyncs/sonnet")

Generating train split: 0 examples [00:00, ? examples/s]

## Andrej uses a names dataset, but I will explore sonnet generation instead

The task will be to generate (possibly Shakespearean-style) sonnets.

## Tokenization

There are a few approahces for tokenization. BPE is the real algorithm an LLm uses. For efficiency (and simplicity) in MicroGPT, I will use character-based tokenization.

In [13]:
train_ds = ds['train']

In [19]:
train_ds[-1]

{'text': "Love's fire heats water, water cools not love."}

In [20]:
unique_chars = set()

for row in train_ds:
  chars_list = list(row['text'])
  for c in chars_list:
    unique_chars.add(c)


In [30]:
# Vocabulary size
print("Vocab size:", len(unique_chars))

## Now convert this list of chars to a proper vocabulary
## char_id -> char
vocab = {index: char for index, char in enumerate(unique_chars)}

## The tokenizer does the opposite: go from text to integers
tokenizer = {char: index for index, char in enumerate(unique_chars)}

Vocab size: 59


In [29]:
vocab

{0: 'X',
 1: 'M',
 2: 'B',
 3: 'b',
 4: 'a',
 5: 'C',
 6: 'F',
 7: 'u',
 8: 'T',
 9: 'A',
 10: '!',
 11: 'r',
 12: 'i',
 13: ',',
 14: 'E',
 15: 'w',
 16: 'z',
 17: 'P',
 18: 'm',
 19: ':',
 20: 's',
 21: 'R',
 22: 'N',
 23: 'L',
 24: 'G',
 25: 'V',
 26: 'K',
 27: 'j',
 28: 'h',
 29: 'y',
 30: ' ',
 31: 'J',
 32: 'q',
 33: 'D',
 34: ';',
 35: 'n',
 36: "'",
 37: 'f',
 38: '-',
 39: 'O',
 40: 'x',
 41: 'o',
 42: 'U',
 43: 'p',
 44: 't',
 45: 'v',
 46: 'I',
 47: 'W',
 48: 'Y',
 49: 'l',
 50: 'S',
 51: 'g',
 52: 'c',
 53: 'd',
 54: '.',
 55: 'k',
 56: 'e',
 57: '?',
 58: 'H'}

## Build Autograd Now

We want all basic operations in the architecture.

In [384]:
class Value:
  def __init__(self, val : float, name="", op="", children=[]):
    self.val = val
    self.name = name
    self.children = children # should be ordered, not sets!
    self.local_grads = [] # local gradient of current node wrt children
    self.grad = 0 # total gradient (add to this)

  def __repr__(self):
    return {self.val}
  
  def __str__(self):
    return (f"{self.name}: {self.val}")


  # Overrie original add function for Value object
  def __add__(self, other, op="+", other_name=None):
    if not isinstance(other, Value):
      other = Value(other)
    new_val = Value(self.val + other.val, children=set([self, other]))
    new_val.local_grads = [1,1]
    return new_val
  
  def __neg__(self):
    new_val = Value(-self.val, children=[self])
    new_val.local_grads = [-1]
    return new_val
  
  def __sub__(self, other):
    return self.__add__((-other), other_name= other.name)
  
  def __mul__(self, other, other_name=None):
    if not isinstance(other, Value):
      other = Value(other)
    new_val = Value(self.val * other.val, children=set([self, other]))
    new_val.local_grads = [other.val, self.val]
    return new_val
  
  def __rmul__(self, other, other_name=None):
    if not isinstance(other, Value):
      other = Value(other)
    new_val = Value(self.val * other.val, children=set([self, other]))
    new_val.local_grads = [other.val, self.val]
    return new_val
  
  def __pow__(self, exp : float, op="^"):
    new_val = Value(self.val ** exp, children=[self])
    new_val.local_grads = [exp * self.val ** (exp - 1)]
    return new_val
  
  def __truediv__(self, other, op="/"):
    if isinstance(other, (int, float)):
      other = Value(float(other))
    new_val = Value(self.val / other.val, children=[self, other])
    new_val.local_grads = [1.0 / other.val, -self.val / (other.val * other.val)]
    return new_val
  
  def exp(self):
    new_val = Value(math.exp(self.val), children=[self])
    new_val.local_grads = [math.exp(self.val)]
    return new_val
  
  def log(self):
    new_val = Value(math.log(self.val), children=[self])
    new_val.local_grads = [1 / self.val]
    return new_val
  
  def relu(self):
    new_val = Value(max(0, self.val), children=[self])
    new_val.local_grads = [float(self.val > 0)]
    return new_val
  
  def backward(self):
    # compute the gradient from the root
    topo = []
    visited = set()
    # first construct a topsort of the computational graph
    def construct_toposort(cur):
      if cur not in visited:
        visited.add(cur)
        for n in cur.children:
          construct_toposort(n)
        topo.append(cur)
    
    construct_toposort(self)
    self.grad = 1
    for v in reversed(topo):
      for child, local_grads in zip(v.children, v.local_grads):
        child.grad += local_grads * v.grad

In [385]:
inf = Value(float('-inf'))
inf

TypeError: __repr__ returned non-string (type set)

In [386]:
a = Value(5, "a")
b = Value(3, "b")

In [311]:
c = a + b
d = c**3

In [271]:
d.backward()

In [272]:
d.grad

1

In [207]:
c.grad

192

In [208]:
a.grad

192

In [209]:
b.grad

192

In [210]:
a = 5
b = 3
c = a + b # 8
d = c^3

so grad(d) = 1
grad (c) = 3 c^2
grad(b) = 

SyntaxError: invalid syntax (4045663001.py, line 6)

In [211]:
a = Value(2.0)
b = Value(3.0)
c = a * b       # c = 6.0
L = c + a       # L = 8.0
L.backward()
print(a.grad)   # 4.0 (dL/da = b + 1 = 3 + 1, via both paths)
print(b.grad)   # 2.0 (dL/db = a = 2)
print(c.grad)   
print(L.grad)   

4.0
2.0
1
1


## Now that AutoGrad is working, can actually define our model!

This won't be too big of a model (just overfit to sonnets), so will have:
- embedding dimension: 16
- attention heads: 4
- layers: 1
- block size: 16 # max sequence length

In [387]:
embed_dim = 16
attn_heads = 4
layers = 1
block_size = 16
hidden_dim = int(embed_dim / attn_heads) # dimension of each head
# define some matrix
vocab_size = len(vocab)
def create_matrix(in_dim, out_dim, std=0.08):
  # creates a matrix of size in_dim x out_dim
  return [[Value(random.gauss(0, std)) for _ in range(out_dim)] for _ in range(in_dim)]

state_dict = {}
for i in range(layers):
  # Projects embedding -> Query vectors
  state_dict[f'layer_{i}_attn_wq'] = create_matrix(embed_dim, embed_dim)
  # Projects embedding -> Key vectors
  state_dict[f'layer_{i}_attn_wk'] = create_matrix(embed_dim, embed_dim)
  # Projects embedding -> Value vectors
  state_dict[f'layer_{i}_attn_wv'] = create_matrix(embed_dim, embed_dim)
  # Final projection after concatenating all heads.
  state_dict[f'layer_{i}_attn_wo'] = create_matrix(embed_dim, embed_dim)
  # Standard MLP: Linear(embed_dim -> 4*embed_dim); GELU; Linear(4*embed_dim -> embed_dim)
  state_dict[f'layer_{i}_fc1'] = create_matrix(embed_dim, 4 * embed_dim)
  state_dict[f'layer_{i}_fc2'] = create_matrix(4 * embed_dim, embed_dim)

# Used to map final hidden state -> logits over vocabulary:
state_dict[f'output_head'] = create_matrix(vocab_size, embed_dim)
# This is positional embedding
state_dict[f'pos_embedding'] = create_matrix(block_size, embed_dim)
# this is token embedding
state_dict[f'token_embedding'] = create_matrix(vocab_size, embed_dim)

params = [p for mat in state_dict.values() for row in mat for p in row]


In [388]:
class MHA:
  def __init__(self, wq, wk, wv, wo, config):
    self.wq = wq
    self.wk = wk
    self.wv = wv
    self.wo = wo
    self.config = config
  
  def forward(self, input, attn_mask):
    attn_weights = (self.wq @ self.wk.T) / math.sqrt(self.config.hidden_dim)

    

    masked_attn_weights = attn_weights +  attn_mask



In [389]:
def linear(W, x):
  return [sum((w_i * x_i for w_i, x_i in zip(w_row, x)), Value(0.0)) for w_row in W]

def softmax(input):
  denom = sum((Value.exp(x) for x in input), Value(0.0))
  logits = [x.exp() / denom for x in input]
  return logits

def rmsnorm(x):
  ms = sum((xi * xi for xi in x), Value(0.0)) / Value(float(len(x)))
  scale = (ms + 1e-5) ** -0.5
  return [xi * scale for xi in x]

In [390]:
attn_mask = [[Value(float('-inf')) for _ in range(hidden_dim)] for _ in range(hidden_dim)]
for i in range(hidden_dim):
  for j in range(i):
    attn_mask[i][j] = 0


In [391]:
def one_hot(dim, i):
  output = [Value(0) for _ in range(dim)]
  output[i] = 1
  return output

In [392]:
def gpt(token_id, pos_id, keys, values): # add attention mask?
  # one_hot_token_id = one_hot(vocab_size, token_id)
  # token_emb = linear(state_dict[f'token_embedding'], one_hot_token_id)

  # one_hot_pos_id = one_hot(block_size, pos_id)
  # pos_emb = linear(state_dict[f'pos_embedding'], one_hot_pos_id)

  token_emb = state_dict['token_embedding'][token_id]
  pos_emb = state_dict['pos_embedding'][pos_id]
  embedding = token_emb + pos_emb
  embedding = rmsnorm(embedding)

  input = embedding
  output = None
  for i in range(layers):
    residual = input
    # MHA
    Q = linear(state_dict[f'layer_{i}_attn_wq'], input)
    K = linear(state_dict[f'layer_{i}_attn_wk'], input)
    V = linear(state_dict[f'layer_{i}_attn_wv'], input)

    keys.append(K)
    values.append(V)

    x_attn = []
    for h in range(attn_heads):
      hs = h * hidden_dim
      q_h = Q[hs: hs + hidden_dim]
      k_h = [ki[hs: hs + hidden_dim] for ki in keys[i]]
      v_h = [vi[hs: hs + hidden_dim] for vi in values[i]]
      attn_logits = [sum(q_h[j] * k_h[t][j] for j in range(hidden_dim)) / math.sqrt(hidden_dim) for t in range(len(k_h))]
      attn_weights = softmax(attn_logits)
      head_out = [sum(attn_weights[i] * v_h[t][j] for t in range(len(v_h))) for j in range(hidden_dim)]

      x_attn.extend(head_out)

    
    x_proj = linear(state_dict[f'layer_{i}_attn_wo'], x_attn)
    x = [x_i + r for x_i, r in zip(x_proj, residual)]
    x_residual = x
    x = rmsnorm(x)

    ## Feed Forward

    l1 =  linear(state_dict[f'layer_{i}_fc1'], x)
    l1_relu = [x.relu() for x in l1]
    l2 = linear(state_dict[f'layer_{i}_fc1'], l1_relu)
    x = l2 + x_residual

    input = x
    output = x
  logits = linear(state_dict['output_head'], output)
  return logits

In [393]:
lr = 0.01
beta_1 = .85
beta_2 = .99
eps_adam = 1e-8

steps = 1000

m1 = [0 for _ in params]
m2 = [0 for _ in params]

for step in range(steps):
  sonnet = train_ds[step]['text']
  tokens = [tokenizer[c] for c in list(sonnet)]
  n = min(block_size, len(tokens) - 1)

  keys, values = [[] for _ in range(layers)], [[] for _ in range(layers)]
  losses = []
  for pos_id in range(n):
    token_id, target_id = tokens[pos_id], tokens[pos_id + 1]
    logits = gpt(token_id, pos_id, keys, values)
    probs = softmax(logits)
    loss_t = -probs[target_id].log() # cross entropy, only count - 1 * log(p)
    losses.append(loss_t)

  loss = sum(losses, Value(0.0)) / Value(float(n))

  loss.backward()

  lr_t = lr * (1 - step / steps)
  for i, p in enumerate(params):
    m1[i] = beta_1 * m1[i] + (1 - beta_1) * p.grad
    m2[i] = beta_2 * m2[i] + (1 - beta_2) * p.grad ** 2

    m1_hat = m1[i] / (1 - beta_1 ** (step + 1))
    m2_hat = m2[i] / (1 - beta_2 ** (step + 1))

    p.val -= lr_t * m1_hat / (m2_hat ** 0.5 + eps_adam) ## fast update

    p.grad = 0 # zero gradients at end 

  print(f"step {step+1:4d} / {steps:4d} | loss {loss.val:.4f}")

step    1 / 1000 | loss 4.0827
step    2 / 1000 | loss 4.0744
step    3 / 1000 | loss 4.0704
step    4 / 1000 | loss 4.0728
step    5 / 1000 | loss 4.0646
step    6 / 1000 | loss 4.0649
step    7 / 1000 | loss 4.0658
step    8 / 1000 | loss 4.0590
step    9 / 1000 | loss 4.0505
step   10 / 1000 | loss 4.0457
step   11 / 1000 | loss 4.0456
step   12 / 1000 | loss 4.0434
step   13 / 1000 | loss 4.0320
step   14 / 1000 | loss 4.0300
step   15 / 1000 | loss 4.0291
step   16 / 1000 | loss 4.0213
step   17 / 1000 | loss 4.0320
step   18 / 1000 | loss 4.0170
step   19 / 1000 | loss 3.9679
step   20 / 1000 | loss 3.9652
step   21 / 1000 | loss 3.9196
step   22 / 1000 | loss 3.9537
step   23 / 1000 | loss 3.9232
step   24 / 1000 | loss 3.8676
step   25 / 1000 | loss 3.9383
step   26 / 1000 | loss 3.8293
step   27 / 1000 | loss 3.7553
step   28 / 1000 | loss 3.7911
step   29 / 1000 | loss 3.8245
step   30 / 1000 | loss 3.5528
step   31 / 1000 | loss 3.6966
step   32 / 1000 | loss 3.6148
step   3

In [350]:
print('vocab_size', vocab_size)
print('target_id', target_id)
print('len(logits)', len(logits))
print('len(probs)', len(probs))

vocab_size 59
target_id 4
len(logits) 59
len(probs) 59


In [401]:
## See if the model overfits with the forward pass

context_text = "Love's f"
inputs = list(context_text)
tokens = [tokenizer[c] for c in list(context_text)]

temp = 0.5

for sample_idx in range(20):
  keys, values = [[] for _ in range(layers)], [[] for _ in range(layers)]
  token_id = 5
  sample = []
  for pos_id in range(block_size):
    logits = gpt(token_id, pos_id, keys, values)
    probs = softmax([l / temp for l in logits])

    token_id = random.choices(range(vocab_size), weights=[p.val for p in probs])[0]

    sample += [token_id]

full_txt = ''.join([vocab[c] for c in sample])

In [402]:
full_txt

'  thaluo annda l'

Yeah so this is complete nonsense. Try again on list of names. That's definitely much easier.